<a href="https://colab.research.google.com/github/amandaay/BigDataAnalytics/blob/main/CS7280proj3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download dataset from kaggle

## feel free to skip this step
### By obtaining csv file from https://storage.googleapis.com/cs7280-proj3/redfin.csv instead. This saves time from downloading the data again.

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

In [ ]:
# Step 4: Set appropriate permissions for the Kaggle API token
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Step 5: Download the dataset
!kaggle datasets download -d vincentvaseghi/us-cities-housing-market-data

 99% 0.99G/1.01G [00:26<00:00, 57.0MB/s]
100% 1.01G/1.01G [00:26<00:00, 40.4MB/s]


In [ ]:
# Step 6: Unzip the dataset
!unzip us-cities-housing-market-data.zip

Archive:  us-cities-housing-market-data.zip
  inflating: city_market_tracker.tsv000  


## Convert tsv file to csv file format

In [ ]:
# Step 7: Import necessary libraries
import pandas as pd

In [ ]:
csv_table = pd.read_table('city_market_tracker.tsv000', sep='\t')

In [ ]:
csv_table.to_csv('redfin.csv', index=False)

KeyboardInterrupt: 

In [ ]:
import os
print(os.getcwd())

In [ ]:
!mv redfin.csv /content/drive/MyDrive

# Install Pyspark and appropriate files

In [ ]:
#@title Install all the appropriate packages

!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
#@title Import and create Spark Context

# Let's import the libraries we will need
import itertools

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
from pyspark import SparkFiles

In [ ]:
from pyspark.sql.functions import year, col

In [ ]:
# URL of the CSV file
url = "https://storage.googleapis.com/cs7280-proj3/redfin.csv"

In [ ]:
sc.addFile(url)

In [ ]:
path = SparkFiles.get("redfin.csv")

In [ ]:
df = spark.read.csv("file://"+ path, header=True, inferSchema=True)

In [ ]:
df.show()

In [ ]:
distinct_states_df = df.select(col("state"), col("state_code")).distinct()

In [ ]:
distinct_states_df.show(distinct_states_df.count(), truncate=False)

In [ ]:
sold_inventory = df.select(year("period_begin").alias("year"), "homes_sold", "inventory", "state")

In [ ]:
sold_inventory = sold_inventory.withColumn("difference", col("inventory") - col("homes_sold"))

In [ ]:
sold_inventory = sold_inventory.na.drop(subset=["difference"])

In [ ]:
sold_inventory.show()

In [ ]:
sold_inventory = sold_inventory.rdd

In [ ]:
sold_inventory.take(5)

In [ ]:
# sort by year and sort by differences
sorted_sold_inventory = sold_inventory.sortBy(lambda x: (x[0], -x[4]))

In [ ]:
sorted_sold_inventory.take(5)

In [ ]:
grouped_by_year_state = sorted_sold_inventory.map(lambda x: ((x[0], x[3]), (x[4], 1))).reduceByKey(lambda x, y: (x[0]+y[0], x[1] + y[1])).mapValues(lambda x: (x[0] / x[1])).map(lambda x:( x[0][0], x[0][1],x[1]))

In [ ]:
grouped_by_year_state.take(10)

In [ ]:
# Take the top 5 differences for each year
top_5_by_year = grouped_by_year_state.groupBy(lambda x: x[0]).flatMapValues(lambda x: sorted(x, key=lambda y: -y[2])[:5]).map(lambda x: (x[0], x[1][1], x[1][2]))

In [ ]:
top_5_by_year.collect()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Define your data
data = top_5_by_year.collect()

# Create DataFrame
df = pd.DataFrame(data, columns=['Year', 'State', 'Value'])

# Pivot the data to have states as columns and years as rows
pivot_df = df.pivot(index='Year', columns='State', values='Value')

# Rank the values in descending order (1 = highest value)
ranked_df = pivot_df.rank(axis=1, ascending=False, method='first')

# Plotting
fig, ax = plt.subplots(figsize=(15, 10))
colors = plt.get_cmap('tab10')

for column in ranked_df.columns:
    line, = ax.plot(ranked_df.index, ranked_df[column], label=column, marker='o', color=colors(ranked_df.columns.tolist().index(column) / len(ranked_df.columns)))
    for idx in ranked_df.index:
        ax.annotate(f'{pivot_df.at[idx, column]:.0f}',
                    xy=(idx, ranked_df.at[idx, column]),
                    textcoords="offset points",
                    xytext=(0, 5),
                    ha='center',
                    fontsize=10,
                    color=line.get_color())

ax.set_xlabel('Year')
ax.set_ylabel('Rank')
ax.set_title('Top five states with biggest inventory surplus from 2012 to 2024', pad=10, fontsize=22)
ax.invert_yaxis()  # Invert y axis so that rank 1 is at the top
ax.legend(title='State')

# Set y-axis to show only integer ticks
ax.yaxis.set_major_locator(plt.MaxNLocator(integer=True))

plt.show()

In [ ]:
spark.stop()